# Stirnu Buks Rating Calibration

This notebook calibrates the parameters of the pace-adjusted TrueSkill model used for the Stirnu Buks leaderboard. The central question is: **how should the dynamic β (beta) be set so that a normalized pace gap between two runners produces the right amount of rating movement?**

The analysis follows a natural progression:
1. Verify that finish-time distributions are not Gaussian → motivates normalization rather than using raw times
2. Confirm that per-race median normalization produces a consistent scale across events → validates the σ estimate
3. Measure the distribution of pairwise pace gaps → calibrates `GAP_MULTIPLIER`
4. Examine participant consistency and repeat-race behaviour → informs `MIN_RACES`

The `YEAR` and `DISTANCE_NAME` settings below control which dataset is analysed throughout.

## Setup

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import scipy.stats as stats

from constants import RACES_FILE_PATH
from explore import RaceExplorer
from models.stirnu_buks import parse_stirnu_buks_to_dataframes

YEAR          = 2025           # set to None to include all years
DISTANCE_NAME = "lūsis"        # options: 'vāvere', 'zaķis', 'stirnu_buks', 'lūsis' — or None for all
TITLE_SUFFIX  = f"({YEAR} - {DISTANCE_NAME})" 

In [ ]:
races_raw = json.loads(RACES_FILE_PATH.read_text())
races_df_full, results_df_full = parse_stirnu_buks_to_dataframes(races_raw.get("stirnu_buks"))

explorer = RaceExplorer(races_df_full, results_df_full)

races_df, results_df = explorer._filter(YEAR, DISTANCE_NAME)
results_df = explorer._compute_pace(races_df, results_df)
results_df = explorer._compute_normalized_pace(results_df)
race_ids   = races_df.index.tolist()

print(f"Races: {len(races_df)}, Results: {len(results_df)}, Unique participants: {results_df['participant'].nunique()}")
print(f"\nAvailable combinations: {explorer.combinations()}")

## Are finish times normally distributed?

TrueSkill treats every head-to-head comparison as a draw from a Gaussian. If finish times are heavily skewed, using raw times for pairwise comparisons would systematically misrepresent competitive differences — especially at the extremes of the field. Before normalizing, we formally test whether each race's time distribution looks Gaussian.

In [ ]:
n_cols = 3
n_rows = int(np.ceil(len(race_ids) / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, n_rows * 4))
axes = axes.flatten()

for ax, race_id in zip(axes, race_ids):
    subset = results_df[results_df['race_id'] == race_id]['pace_s'].dropna()
    race_name = f"{races_df.loc[race_id, 'name']} #{races_df.loc[race_id, 'event_no']}"

    ax.hist(subset / 60, bins=20, density=True, alpha=0.6, color='steelblue', label='Finish time')

    mu, std = stats.norm.fit(subset / 60)
    x = np.linspace((subset / 60).min(), (subset / 60).max(), 100)
    ax.plot(x, stats.norm.pdf(x, mu, std), 'r-', lw=2, label=f'Normal μ={mu:.1f} σ={std:.1f}')

    if len(subset) >= 8:
        _, p = stats.shapiro(subset)
        ax.set_title(f"{race_name}\nShapiro p={p:.3f} {'✓ normal' if p > 0.05 else '✗ not normal'}")
    else:
        ax.set_title(race_name)

    ax.set_xlabel('Finish time (min)')
    ax.legend(fontsize=7)

for ax in axes[len(race_ids):]:
    ax.set_visible(False)

plt.suptitle(f'Finish Time Distributions per Race {TITLE_SUFFIX}', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

Every race rejects the Shapiro-Wilk test (p ≪ 0.05). The histograms confirm a consistent right skew: the back of the field stretches much further from the median than the front does. This is expected in trail running — tired runners slow down more than fresh runners speed up. Raw finish times cannot safely be used in TrueSkill comparisons, which motivates normalizing relative to the race median.

## Skew: mean vs median pace

The clearest symptom of right skew is mean > median. We check this per race as a sanity check before normalizing — if the skew is consistent in direction and roughly stable in magnitude across events, a single normalization scheme (dividing by the race median) should put all races on a comparable scale.

In [ ]:
agg = results_df.groupby('race_id')['pace'].agg(['mean', 'median', 'std'])
agg['diff_pct'] = (agg['mean'] - agg['median']) / agg['median'] * 100
agg['label'] = agg.index.map(lambda i: f"{races_df.loc[i, 'name']} #{races_df.loc[i, 'event_no']}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

x = np.arange(len(agg))
ax1.bar(x - 0.2, agg['mean'], 0.4, label='Mean', color='steelblue', alpha=0.8)
ax1.bar(x + 0.2, agg['median'], 0.4, label='Median', color='coral', alpha=0.8)
ax1.set_xticks(x)
ax1.set_ylim(bottom=300)
ax1.set_xticklabels(agg['label'], rotation=45, ha='right')
ax1.set_ylabel('Pace (s/km)')
ax1.set_title('Mean vs Median pace')
ax1.legend()

ax2.bar(x, agg['diff_pct'], color=['green' if v >= 0 else 'red' for v in agg['diff_pct']], alpha=0.8)
ax2.axhline(0, color='black', linewidth=0.8)
ax2.set_xticks(x)
ax2.set_xticklabels(agg['label'], rotation=45, ha='right')
ax2.set_ylabel('(Mean - Median) / Median %')
ax2.set_title('Skew: positive = right tail (slow runners)')

plt.tight_layout()
plt.show()

print(agg[['label', 'mean', 'median', 'diff_pct']].to_string())

Mean pace consistently exceeds median in every race, with the gap ranging from roughly 3–8%. The skew is stable in direction and magnitude across events, confirming that per-race median normalization is appropriate.

## Normalized pace

Raw finish times cannot be compared across races — terrain, distance, and conditions differ each time. We define normalized pace as deviation from the race median:

`norm_pace = (pace − median_pace) / median_pace`

A value of 0 means exactly median speed; +0.1 means 10% slower than median. The spread (σ) of this distribution directly informs TrueSkill's β. The skewness stats below confirm that normalization preserves the right-skew character (the distribution is not made symmetric), but the scale is now race-agnostic.

In [ ]:
from scipy.stats import skew, skewtest, kurtosis

for race_id in race_ids:
    subset = results_df[results_df['race_id'] == race_id]['norm_pace'].dropna()
    sk = skew(subset)
    kurt = kurtosis(subset)  # excess kurtosis, 0 = normal
    _, p = skewtest(subset)  # tests if skew is significantly different from 0
    race_name = f"{races_df.loc[race_id, 'name']} #{races_df.loc[race_id, 'event_no']}"
    print(f"{race_name}: skew={sk:.3f}, kurtosis={kurt:.3f}, skewtest p={p:.3f}")

## Normalized pace distribution

With normalization defined, we inspect its distribution — both pooled across all races and per race. This answers two questions: (a) does normalization produce a consistent spread across events, and (b) what σ characterizes the typical performance range that β needs to account for?

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

axes[0].hist(results_df['norm_pace'], bins=40, color='steelblue', alpha=0.7, density=True)
mu, std = stats.norm.fit(results_df['norm_pace'].dropna())
x = np.linspace(results_df['norm_pace'].min(), results_df['norm_pace'].max(), 200)
axes[0].plot(x, stats.norm.pdf(x, mu, std), 'r-', lw=2, label=f'Normal fit μ={mu:.3f} σ={std:.3f}')
axes[0].set_xlabel('Normalized pace')
axes[0].set_title(f'Normalized pace distribution {TITLE_SUFFIX}')
axes[0].legend()

data_per_race = [results_df[results_df['race_id'] == rid]['norm_pace'].dropna().values for rid in race_ids]
labels = [f"#{races_df.loc[rid, 'event_no']}" for rid in race_ids]
axes[1].boxplot(data_per_race, labels=labels)
axes[1].axhline(0, color='red', linestyle='--', alpha=0.5, label='Median (=0 by definition)')
axes[1].set_xlabel('Race')
axes[1].set_ylabel('Normalized pace')
axes[1].set_title('Normalized pace spread per race')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"Overall norm_pace std: {results_df['norm_pace'].std():.4f}")

The pooled σ ≈ 0.20, meaning a typical runner's pace falls within roughly ±20% of the race median. The per-race boxplots show similar interquartile ranges across all events, confirming that normalization makes the performance scale comparable across races. We treat σ ≈ 0.20 as the reference spread when reasoning about β.

## Pairwise pace gaps: calibrating the dynamic β multiplier

The pace-adjusted TrueSkill update uses a dynamic β: `β = max(β_min, β_base / (1 + gap × M))`. A large normalized pace gap → smaller β → the faster runner is expected to win decisively → the update moves ratings less. We want β to halve at a *typical* competitive gap — sensitive enough to distinguish genuine performance differences, but not so aggressive that it suppresses all rating movement from close finishes. To pick the multiplier M, we need the empirical distribution of pairwise normalized pace gaps.

In [ ]:
gaps, percentiles = explorer._compute_pairwise_gaps(races_df, results_df)

fig, ax = plt.subplots(figsize=(12, 5))
ax.hist(gaps, bins=60, color='steelblue', alpha=0.7, density=True)
for p, v in percentiles.items():
    ax.axvline(v, linestyle='--', label=f'p{p}={v:.3f}')
ax.set_xlabel('Pairwise normalized pace gap')
ax.set_title(f'Distribution of pairwise pace gaps {TITLE_SUFFIX}')
ax.legend()
plt.tight_layout()
plt.show()

print(f"Median pairwise gap : {percentiles[50]:.4f}")
print(f"p75 gap             : {percentiles[75]:.4f}")
print(f"p90 gap             : {percentiles[90]:.4f}")
print(f"Suggested M         : 1 / {percentiles[50]:.4f} = {1/percentiles[50]:.1f}")

The median pairwise gap is ≈ 0.18 normalized pace units. Setting `M ≈ 1/0.18 ≈ 5.5` makes β halve at the typical competitive gap, matching `GAP_MULTIPLIER = 6` in `constants.py`. The long right tail (gaps > 0.5) covers front-to-back matchups that are highly one-sided — these receive a very small β and contribute minimal rating movement, which is the intended behaviour for large mismatches.

## Field composition: repeat participants

TrueSkill learns from repeated matchups between the same runners. A participant who appears only once stays near their prior (μ = 25, σ = 8.33) and their rating carries little information. The `MIN_RACES` threshold sets the minimum evidence required before a rating is considered meaningful — too low pollutes the leaderboard with under-tested entries; too high excludes most of the field. We profile how many participants return across multiple races.

In [ ]:
composition = explorer._compute_field_composition(results_df)
race_counts_per_participant = composition['race_counts']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(race_counts_per_participant, bins=range(1, race_counts_per_participant.max() + 2),
             color='steelblue', alpha=0.8, align='left')
axes[0].set_xlabel('Number of races entered')
axes[0].set_ylabel('Number of participants')
axes[0].set_title('Participant race frequency')
axes[0].xaxis.set_major_locator(mticker.MultipleLocator(1))

thresholds = range(1, race_counts_per_participant.max() + 1)
coverage = [(race_counts_per_participant >= t).sum() / race_counts_per_participant.count() * 100 for t in thresholds]
axes[1].plot(list(thresholds), coverage, marker='o', color='coral')
axes[1].set_xlabel('Min races threshold')
axes[1].set_ylabel('% of participants included')
axes[1].set_title('Leaderboard coverage vs MIN_RACES threshold')
axes[1].xaxis.set_major_locator(mticker.MultipleLocator(1))
axes[1].yaxis.set_major_formatter(mticker.PercentFormatter())
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(race_counts_per_participant.value_counts().sort_index().rename('participants').to_frame())

About 51% of participants appear in 2+ races, and 34% in 3+. In a 5-race season, `MIN_RACES = 2` retains roughly half the field while excluding single-race entrants whose ratings are untested. Raising to 3 drops coverage by another ~18 percentage points for limited benefit — most of the rating signal comes from the first two appearances anyway.

## Gap structure: does the field spread out toward the back?

Our dynamic β applies the same formula regardless of finishing position. If gaps systematically grow toward the back — slower runners more dispersed than fast runners — then a given absolute gap means different things depending on where in the field it occurs. We check whether the step from one finisher to the next varies by position.

In [ ]:
gap_struct = explorer._compute_gap_structure(races_df, results_df)

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

all_consec = []
for race_id in race_ids:
    subset = results_df[results_df['race_id'] == race_id].sort_values('pace')['norm_pace'].dropna().values
    all_consec.extend(np.diff(subset))

axes[0].hist(np.array(all_consec), bins=40, color='steelblue', alpha=0.7)
axes[0].set_xlabel('Normalized pace gap to next finisher')
axes[0].set_title('Consecutive finisher gaps (all races)')

pos_pcts, gap_vals = [], []
for race_id in race_ids:
    subset = results_df[results_df['race_id'] == race_id].sort_values('pace')['norm_pace'].dropna().values
    n = len(subset)
    for i, gap in enumerate(np.diff(subset)):
        pos_pcts.append(i / n * 100)
        gap_vals.append(gap)

axes[1].scatter(pos_pcts, gap_vals, alpha=0.2, s=10, color='steelblue')
trend = np.poly1d(np.polyfit(pos_pcts, gap_vals, 1))
xs = np.linspace(0, 100, 100)
axes[1].plot(xs, trend(xs), 'r-', lw=2, label=f"Trend slope={gap_struct['trend_slope']:.5f}")
axes[1].set_xlabel('Finishing position percentile')
axes[1].set_ylabel('Gap to next finisher (norm pace)')
axes[1].set_title('Do gaps grow toward the back?')
axes[1].legend()

plt.suptitle(f'Gap Structure {TITLE_SUFFIX}', fontsize=14)
plt.tight_layout()
plt.show()

Gaps do grow toward the back, but only mildly. The dynamic β formula already handles this naturally: larger gaps → smaller β → more decisive outcome for the model. No additional position-dependent adjustment is needed.

## Participant pace consistency across races

A key assumption of any rating system is that a runner's performance is a *stable* signal — someone who is fast in one race should tend to be fast in others. If normalized pace varies wildly across appearances, TrueSkill cannot converge on meaningful ratings regardless of how β is tuned. We check this by examining how consistently repeat participants perform.

In [ ]:
repeat_participants = results_df.groupby('participant').filter(lambda x: len(x) >= 2)

consistency = repeat_participants.groupby('participant')['norm_pace'].agg(['std', 'mean', 'count'])
consistency.columns = ['pace_std', 'pace_mean', 'n_races']

ordered = results_df.sort_values('race_id')
first_pace  = ordered.groupby('participant')['norm_pace'].nth(0).rename('pace_r1')
second_pace = ordered.groupby('participant')['norm_pace'].nth(1).rename('pace_r2')
paired = first_pace.to_frame().join(second_pace, how='inner').dropna()
corr = paired['pace_r1'].corr(paired['pace_r2'])

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

axes[0].hist(consistency['pace_std'].dropna(), bins=30, color='steelblue', alpha=0.7)
axes[0].axvline(consistency['pace_std'].median(), color='red', linestyle='--',
                label=f"Median std = {consistency['pace_std'].median():.3f}")
axes[0].set_xlabel('Std of normalized pace across races')
axes[0].set_ylabel('Participants')
axes[0].set_title(f'Per-participant pace consistency {TITLE_SUFFIX}')
axes[0].legend()

axes[1].scatter(paired['pace_r1'], paired['pace_r2'], alpha=0.4, s=20, color='steelblue')
m, b = np.polyfit(paired['pace_r1'], paired['pace_r2'], 1)
xs = np.linspace(paired['pace_r1'].min(), paired['pace_r1'].max(), 100)
axes[1].plot(xs, m * xs + b, 'r-', lw=2, label=f'r = {corr:.2f}')
axes[1].set_xlabel('Normalized pace — first race')
axes[1].set_ylabel('Normalized pace — next race')
axes[1].set_title(f'Race-to-race pace correlation {TITLE_SUFFIX}')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"Median per-participant pace std : {consistency['pace_std'].median():.3f}  (vs overall field std ≈ 0.20)")
print(f"Race-to-race pace correlation   : {corr:.3f}")

A positive race-to-race correlation (r > 0) means a runner's pace in their first race predicts subsequent appearances — there is a real, learnable signal. If the median per-participant std is substantially lower than the overall field std (≈ 0.20), it confirms that between-runner differences are larger than within-runner variation: TrueSkill is measuring something stable rather than just noise.

## Summary: suggested parameter values

Pulling together the findings above into concrete recommendations for `constants.py`.

In [ ]:
result = explorer.analyze(YEAR, DISTANCE_NAME)

print("=== Suggested TrueSkill parameters ===")
print(f"β_base = 4.167  (TrueSkill default μ/6 = 25/6)")
print(f"β_min  = 0.5    (floor for large gaps)")
print()
print(f"=== Dynamic beta calibration ===")
print(f"norm_pace std              : {result.norm_pace_std:.4f}")
print(f"median pairwise gap        : {result.pairwise_median_gap:.4f}")
print(f"suggested GAP_MULTIPLIER   : {result.suggested_multiplier}")
print(f"  → β = max(0.5, 4.167 / (1 + gap × {result.suggested_multiplier}))")
print()
print(f"=== Participant signal quality ===")
print(f"race-to-race correlation   : {result.race_to_race_corr:.3f}")
print(f"median participant std     : {result.median_participant_std:.4f}")
print()
print(f"=== MIN_RACES coverage ===")
print(f"  2+ races: {result.repeat_rate_2plus:.1%} of participants")
print(f"  3+ races: {result.repeat_rate_3plus:.1%} of participants")
print(f"  4+ races: {result.repeat_rate_4plus:.1%} of participants")

## Cross-distance comparison

Run the full analysis for every (year, distance) combination present in the dataset and compare the suggested β multipliers side by side. This is the main automation loop — any distance or year with enough data will be calibrated automatically.

In [ ]:
all_results = explorer.run_all()
explorer.summary_table(all_results)